In [33]:
import pandas as pd

In [34]:
data = pd.read_csv('./SchemeData1908231158SS.csv')

In [35]:
data.head()

,AMC,Code,Scheme Name,Scheme Type,Scheme Category,Scheme NAV Name,Scheme Minimum Amount,Launch Date,Closure Date,ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment
0,Aditya Birla Sun Life AMC Limited,100033,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,5000,24-Feb-1995,24-Feb-1995,INF209K01165
1,Aditya Birla Sun Life AMC Limited,100034,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -R...,5000,24-Feb-1995,24-Feb-1995,INF209K01157INF209K01CE5
2,Aditya Birla Sun Life AMC Limited,119433,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -D...,5000,24-Feb-1995,24-Feb-1995,INF209K01VG0INF209K01VI6
3,Aditya Birla Sun Life AMC Limited,119436,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,5000,24-Feb-1995,24-Feb-1995,INF209K01VH8
4,Aditya Birla Sun Life AMC Limited,100037,Aditya Birla Sun Life Income Fund,Open Ended,Debt Scheme - Medium to Long Duration Fund,Aditya Birla Sun Life Income Fund - Regular - ...,5000,01-Jan-1994,01-Jan-1994,INF209K01587INF209K01CY3


In [36]:
bubbleData = pd.DataFrame({'AMC': data['AMC'], "AMFICode": data['Code'], 'Scheme Name': data['Scheme Name'], 'Scheme Type': data['Scheme Type'], 'AMFI Scheme Category': data['Scheme Category'], 'Scheme NAV Name': data['Scheme NAV Name']})

In [37]:
def scheme_category(scheme):
    list_ = [i.replace("Scheme","").replace("&", "and").strip() for i in scheme.split(' - ')]
    if len(list_) < 2:
        return ["", list_[0]]
    return list_

In [38]:
bubbleData[["Scheme Category", "Sub Category"]] = bubbleData['AMFI Scheme Category'].apply(scheme_category).apply(pd.Series)

In [39]:
bubbleData['Fundtype'] = bubbleData['Sub Category'].apply(lambda x: 'FoF' if 'FoF' in x else 'DE')

In [40]:
bubbleData['PlanType'] = bubbleData['Scheme NAV Name'].apply(lambda x: 'Direct' if 'direct' in x.lower() else 'Regular')

In [41]:
bubbleData['PlanType'].value_counts()

Regular    8083
Direct     6852
Name: PlanType, dtype: int64

In [42]:
def ISIN(x):
    isin_payout = ""
    isin_reinv = ""
    if '-' in x:
        x = x.split('-')
    else:
        x = x.split(' ')
    x = [i.strip() for i in x if len(i)>=12]
    if len(x)==1:
        x=x[0]
        if len(x)==24:
            isin_payout = x[0:12]
            isin_reinv = x[12:24]
        elif len(x)==12:
            isin_payout = x
    elif len(x)==2:
        if len(x[0]) == 12:
            isin_payout = x[0]
        if len(x[1]) == 12:
            isin_reinv = x[1]
    return [isin_payout, isin_reinv]

    return x

In [43]:
bubbleData[["ISIN Div Payout / ISIN Growth", "ISIN Div Reinvestment"]] = data['ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment'].apply(str).apply(ISIN).apply(pd.Series)

In [44]:
bubbleData['Launch Date'] = data['Launch Date']
bubbleData['Closure Date'] = data[' Closure Date']

In [45]:
def amount(x):
    min_amount = ''
    x+=' '
    for i in range(len(x)):
        if x[i].isdigit():
            for k in x[i:]:
                if k.isdigit():
                    min_amount += k
                else:
                    return int(min_amount)
    return None

In [46]:
bubbleData['AMFI-Mininmum-Amount'] = data['Scheme Minimum Amount']
bubbleData['Scheme Minimum Amount'] = data['Scheme Minimum Amount'].apply(str).apply(amount)

In [47]:
bubbleData['isPrimary'] = bubbleData['Scheme NAV Name'].apply(lambda x: 'Y' if 'growth' in x.lower() or 'regular' in x.lower() else 'N')

In [48]:
bubbleData.to_csv('Scheme-Master.csv', index=False)